<a href="https://colab.research.google.com/github/kerryback/ClassicTests/blob/main/fama_french/ff_costequity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import statsmodels.api as sm
!pip install --upgrade pandas-datareader  # need to upgrade to read Yahoo
from pandas_datareader import DataReader as pdr

# read annual original Fama-French factors from 1926 and compute means
ff= pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]
fprem = ff[['Mkt-RF','SMB','HML']].mean()

# read CMA and RMW from 1964, compute means, and combine with other means
ff= pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=1964)[1]
fprem = pd.concat((fprem,ff[['RMW','CMA']].mean()))

# read monthly Fama-French factors for past 60 months
ff = pdr('F-F_Research_Data_5_Factors_2x3','famafrench',start=2017)[0]
ff = ff.iloc[-60:]

# read current 10-year Treasury yield from FRED
rf = pdr('DGS10','fred',start=2022).iloc[-1].item()

In [8]:
# example ticker
ticker = 'CVX'

# compute monthly returns from Yahoo data
# put returns in percent form to match French and FRED data
data = pdr(ticker,'yahoo',start=2016)
close = data['Adj Close'].resample('M').last()
ret = 100*close.pct_change()
ret.name = 'ret'

# merge with French data, reducing to 60 months in the process
ret.index = ret.index.to_period('M')
df = ff.join(ret,how='left')

# compute excess returns for the ticker 
df['ret'] -= df.RF

# run regression of excess return on market excess return and get beta
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
result = sm.OLS(df.ret,sm.add_constant(df[factors])).fit()
betas = result.params[factors]

# compute cost of equity
print(ticker,'cost of equity is',rf+betas@fprem)

CVX cost of equity is 18.264007055879954
